In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import re
import optuna
from optuna import Trial, visualization
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_predict,cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression


# Load train and test data
train_data = pd.read_csv('/kaggle/input/primal2-0/final_train.csv')
test_data = pd.read_csv('/kaggle/input/primal2-0/final_test.csv')

# Concatenate train and test data for preprocessing
combined_data = pd.concat([train_data, test_data], ignore_index=True)

value_mapping = {
    'a': 6,  # Example: 10 lakhs
}
# Replace values in the 'ticket size' column using the mapping
combined_data['Previous_Organizations'] = test_data['Previous_Organizations'].replace(value_mapping)

# Drop specified columns
combined_data.drop(columns=['Designation', 'DOJ', 'R_District', 'B_District', 'Company_1', 'Company_2', 'Skill'], inplace=True)

# Replace Graduation and Qualification values
combined_data['Graduation'] = combined_data['Graduation'].fillna(0).map({'full time': 1, 'part time': 0, 0: 0})
combined_data['Qualification'] = combined_data['Qualification'].map({'graduate': 2, 'under graduate': 1, 'post graduate': 3, 'others': 0, 'diploma holders': 4}).fillna(-1)

# Set 'Previous_Organizations' as type int, treating NaN as 0
combined_data['Previous_Organizations'] = combined_data['Previous_Organizations'].fillna(0).astype(int)

# One-hot encoding for categorical variables
columns_to_encode = ['Industry', 'Source', 'Department', 'R_Region', 'B_Region']
combined_data_encoded = pd.get_dummies(combined_data, columns=columns_to_encode)

# Create indicator variables for states
states = list(combined_data['R_State']) + list(combined_data['B_State'])
for state in states:
    combined_data_encoded[state] = ((combined_data['R_State'] == state) | (combined_data['B_State'] == state)).astype(int)

# Drop 'R_State' and 'B_State'
combined_data_encoded.drop(columns=['R_State', 'B_State', 'Unnamed: 0'], inplace=True)
combined_data_encoded.columns = [re.sub(r'\W+', '_', col) for col in combined_data_encoded.columns]

# Separate back into train and test data
train_data_processed = combined_data_encoded.iloc[:len(train_data)]
test_data_processed = combined_data_encoded.iloc[len(train_data):].drop(columns='Performance')  # Assuming 'Target' is the target column

In [2]:
df_all = train_data_processed
df_test = test_data_processed
df_submission = pd.read_csv("/kaggle/input/sample/sample.csv")

In [3]:
X = df_all.drop(labels=['CandidateID','Performance'],axis=1,inplace=False).copy()
y = df_all['Performance'].values
X_test = df_test.drop(labels=['CandidateID'],axis=1,inplace=False).copy()
random_seed = 0

In [4]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(pd.concat([X,X_test]))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [5]:
df_all['Performance'].value_counts()

Performance
0.0    450
1.0    295
Name: count, dtype: int64

In [6]:
X = df_all.drop(labels=['CandidateID','Performance'],axis=1,inplace=False).copy()
y = df_all['Performance'].values
X_test = df_test.drop(labels=['CandidateID'],axis=1,inplace=False).copy()
random_seed = 0

In [7]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(pd.concat([X,X_test]))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [8]:
from sklearn.metrics import f1_score

def objective(trial, data=X, target=y):
    param_space = {
        'C': trial.suggest_float('C', 1e-3, 1e2, log = True),
        'penalty': trial.suggest_categorical('penalty', ['l2']),
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'sag']),
        'multi_class': trial.suggest_categorical('multi_class', ['ovr', 'multinomial']),
        'max_iter': 10000,
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'tol' : trial.suggest_float('C', 1e-5, 1e-1, log = True),
    }
    
    k = 5
    seed_list = [0]
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=random_seed)
    oof = np.zeros((len(df_all), 2))
    score_list = []
    fold = 1
    
    splits = list(kf.split(X, y))
    for train_idx, val_idx in splits:
        X_train, X_val = X[train_idx, :], X[val_idx, :]
        y_train, y_val = y[train_idx], y[val_idx]
        
        val_preds_list = []
        
        for seed in seed_list:
            param_space['random_state'] = seed
            model = LogisticRegression(**param_space)
            model.fit(X_train, y=y_train)
            val_preds_list.append(model.predict_proba(X_val))
            
        oof[val_idx] = np.mean(val_preds_list, axis=0)
        # Calculate F1 score instead of log loss
        score = f1_score(y_val, np.argmax(oof[val_idx], axis=1), average='weighted')
        print(f"fold: {fold}, F1 score: {score}")
        score_list.append(score)
        fold += 1
    
    cv_f1_score = np.mean(score_list)
    
    return cv_f1_score

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials= 200)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-04-11 02:20:22,712] A new study created in memory with name: no-name-cf0179f4-b08a-4ca6-9f2a-6a2dc9dfe5ce
/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.001, 'high': 100.0, 'log': True}
  warnings.warn(


fold: 1, F1 score: 0.6364509131229797
fold: 2, F1 score: 0.5813491978786993
fold: 3, F1 score: 0.5973154362416108


[I 2024-04-11 02:20:23,127] Trial 0 finished with value: 0.6035823891892196 and parameters: {'C': 3.0750807843227044, 'penalty': 'l2', 'solver': 'newton-cg', 'multi_class': 'ovr', 'class_weight': None}. Best is trial 0 with value: 0.6035823891892196.
/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.001, 'high': 100.0, 'log': True}
  warnings.warn(
[I 2024-04-11 02:20:23,193] Trial 1 finished with value: 0.49136875091257304 and parameters: {'C': 28.576918510129694, 'penalty': 'l2', 'solver': 'sag', 'multi_class': 'ovr', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.6035823891892196.


fold: 4, F1 score: 0.6251816102355279
fold: 5, F1 score: 0.5776147884672802
fold: 1, F1 score: 0.23899914241320722
fold: 2, F1 score: 0.5427224250872695
fold: 3, F1 score: 0.6125472994797906
fold: 4, F1 score: 0.4632340260016659
fold: 5, F1 score: 0.599340861580932


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.001, 'high': 100.0, 'log': True}
  warnings.warn(


...

fold: 5, F1 score: 0.621541485909783
fold: 1, F1 score: 0.6081402075366413
fold: 2, F1 score: 0.5676229420714437
fold: 3, F1 score: 0.6168716028184307


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.001, 'high': 100.0, 'log': True}
  warnings.warn(
[I 2024-04-11 02:21:17,744] Trial 197 finished with value: 0.5984506040177624 and parameters: {'C': 9.323764660388434, 'penalty': 'l2', 'solver': 'lbfgs', 'multi_class': 'multinomial', 'class_weight': None}. Best is trial 3 with value: 0.6240929622892224.
/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than o

fold: 4, F1 score: 0.6425712877965243
fold: 5, F1 score: 0.5570469798657718
fold: 1, F1 score: 0.45491561596136026
fold: 2, F1 score: 0.5351910339079703
fold: 3, F1 score: 0.6216334001568902
fold: 4, F1 score: 0.6313923567252253
fold: 5, F1 score: 0.6083095818674522


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:676: RuntimeWarning: Inconsistent parameter values for distribution with name "C"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'step': None, 'low': 0.001, 'high': 100.0, 'log': True}
  warnings.warn(
[I 2024-04-11 02:21:17,953] Trial 199 finished with value: 0.5285359063982769 and parameters: {'C': 15.710217485915916, 'penalty': 'l2', 'solver': 'sag', 'multi_class': 'multinomial', 'class_weight': 'balanced'}. Best is trial 3 with value: 0.6240929622892224.


fold: 1, F1 score: 0.45175117026676437
fold: 2, F1 score: 0.5317238623782644
fold: 3, F1 score: 0.4700261312578723
fold: 4, F1 score: 0.6140431720266276
fold: 5, F1 score: 0.5751351960618558
Number of finished trials: 200
Best trial: {'C': 13.643544703015806, 'penalty': 'l2', 'solver': 'lbfgs', 'multi_class': 'multinomial', 'class_weight': None}


In [10]:
params = study.best_params
model = LogisticRegression(**params)
model.fit(X, y=y)
pred = model.predict_proba(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
df_test[['class1','class2']] = pred

In [12]:
submission = df_test[['CandidateID','class1','class2']]
submission.to_csv('logistic.csv')